# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [153]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nhl_correlation(): 
    
    ##### READING DATA #####
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    ##### CLEANING CITIES DATA #####
    
    # Removing special characters
    cities.replace('\[([^)]*)\]', '', regex=True, inplace=True)
    
    # Including ' ' into team names
    cities['NHL'] = cities['NHL'].apply(lambda x: ' '.join(re.compile('[A-Z][a-z]+').findall(x)))
    
    # Setting cities as index
    cities.set_index('Metropolitan area', inplace=True)
    
    # Converting str to int
    cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype(int)

    
    ##### CLEANING NHL #####
    
    # Removing special character
    nhl_df['team'].replace('\*', '', regex=True, inplace=True)
    
    # Filtering data
    nhl_df = nhl_df.loc[nhl_df['year'] == 2018]
    nhl_df = nhl_df[nhl_df['GP'].str.isnumeric()]
    
    # Creating Metropolitan area column
    nhl_df['Metropolitan area'] = nhl_df['team'].str.split(' ', expand=True)[0]
    Metropolitan_area_dict = {'Anaheim':'Los Angeles',
                              'Arizona':'Phoenix',
                              'Carolina':'Raleigh',
                              'Colorado':'Denver',
                              'Dallas':'Dallas–Fort Worth',
                              'Florida':'Miami–Fort Lauderdale',
                              'Los':'Los Angeles',
                              'Minnesota':'Minneapolis–Saint Paul',
                              'New':'New York City',
                              'San':'San Francisco Bay Area',
                              'St.':'St. Louis',
                              'Tampa':'Tampa Bay Area',
                              'Vegas':'Las Vegas',
                              'Washington':'Washington, D.C.'}
    
    nhl_df['Metropolitan area'].replace(Metropolitan_area_dict, inplace=True)
    
    # Converting str to int
    nhl_df['W'] = nhl_df['W'].astype(int)
    nhl_df['L'] = nhl_df['L'].astype(int)

    # Creating measure NHL W/L ratio
    nhl_df['NHL W/L'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

    # Grouping by data in a Series with Metropolitan area index
    NHL = nhl_df.groupby('Metropolitan area')['NHL W/L'].mean()
    
    
    ##### MERGING SERIES TO DATAFRAME #####
    
    df = cities.merge(NHL, left_index=True, right_index=True)
    
    
    ##### SELECTING DATA FOR PEARSONR FUNCTION #####
    
    population_by_region = df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['NHL W/L'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [154]:
nhl_correlation()

0.012486162921209909

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [151]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    
    ##### READING DATA #####
    
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    
    ##### CLEANING CITIES DATA #####
    
    # Removing special characters
    cities.replace('\[([^)]*)\]', '', regex=True, inplace=True)
    
    # Including ' ' into team names
    cities['NHL'] = cities['NHL'].apply(lambda x: ' '.join(re.compile('[A-Z][a-z]+').findall(x)))
    
    # Setting cities as index
    cities.set_index('Metropolitan area', inplace=True)
    
    # Converting str to int
    cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype(int)
    
    
    ##### CLEANING NBA DATA #####
    
    # Removing special character
    nba_df['team'].replace('\*', '', regex=True, inplace=True)
    nba_df['team'].replace('\(([^\)]+)\)', '', regex=True, inplace=True)
    
    # Filtering data
    nba_df = nba_df.loc[nba_df['year'] == 2018]
    
    # Converting str to int
    nba_df['W'] = nba_df['W'].astype(int)
    nba_df['L'] = nba_df['L'].astype(int)

    # Creating Metropolitan area column
    lst = nba_df['team']
    lst = lst.str.split(' ')

    for e,i in enumerate(lst):
        if len(i) > 2:
            lst[e] = i[0] + ' ' + i[1]
        else:
            lst[e] = i[0]

    nba_df['Metropolitan area'] = lst

    Metropolitan_area_dict = {'Brooklyn':'New York City',
                              'Dallas':'Dallas–Fort Worth',
                              'Golden State':'San Francisco Bay Area',
                              'Indiana':'Indianapolis',
                              'Dallas':'Dallas–Fort Worth',
                              'Miami':'Miami–Fort Lauderdale',
                              'Minnesota':'Minneapolis–Saint Paul',
                              'New York':'New York City',
                              'Utah':'Salt Lake City',
                              'Portland Trail':'Portland',
                              'Washington':'Washington, D.C.'}

    nba_df['Metropolitan area'].replace(Metropolitan_area_dict, inplace=True)
    
    # Creating measure NBA ratio
    nba_df['NBA W/L'] = nba_df['W'] / (nba_df['W'] + nba_df['L'])
    
    # Grouping by data in a Series with Metropolitan area index
    NBA = nba_df.groupby('Metropolitan area')['NBA W/L'].mean()
    
    
    ##### MERGING SERIES TO DATAFRAME #####
    
    df = cities.merge(NBA, left_index=True, right_index=True)
    
    
    ##### SELECTING DATA FOR PEARSONR FUNCTION #####
    
    population_by_region = df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['NBA W/L'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]



In [152]:
nba_correlation()

-0.17657160252844614

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [156]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    
    ##### READING DATA #####
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    
    ##### CLEANING CITIES DATA #####
    
    # Removing special characters
    cities.replace('\[([^)]*)\]', '', regex=True, inplace=True)
    
    # Including ' ' into team names
    cities['NHL'] = cities['NHL'].apply(lambda x: ' '.join(re.compile('[A-Z][a-z]+').findall(x)))
    
    # Setting cities as index
    cities.set_index('Metropolitan area', inplace=True)
    
    # Converting str to int
    cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype(int)
    
    
    ##### CLEANING MLB #####
    
    # Filtering data
    mlb_df = mlb_df.loc[mlb_df['year'] == 2018]
    
    # Creating Metropolitan area column
    lst = mlb_df['team'].str.split(' ')

    for e,item in enumerate(lst):
        if len(item) > 2:
            lst[e] = item[0] + ' ' + item[1]
        else:
            lst[e] = item[0]
    
    mlb_df['Metropolitan area'] = lst

    Metropolitan_area_dict = {'San Francisco':'San Francisco Bay Area',
                              'Minnesota':'Minneapolis–Saint Paul',
                              'Toronto Blue':'Toronto',
                              'Miami':'Miami–Fort Lauderdale',
                              'Chicago White':'Chicago',
                              'Tampa Bay':'Tampa Bay Area',
                              'Washington':'Washington, D.C.',
                              'Boston Red':'Boston',
                              'Texas':'Dallas–Fort Worth',
                              'Arizona':'Phoenix',
                              'Colorado':'Denver',
                              'New York':'New York City',
                              'Oakland':'San Francisco Bay Area'}

    mlb_df['Metropolitan area'].replace(Metropolitan_area_dict, inplace=True)
    
    # Creating measure MLB ratio
    mlb_df['MLB W/L'] = mlb_df['W'] / (mlb_df['W'] + mlb_df['L'])
    
    # Grouping by data in a Series with Metropolitan area index
    MLB = mlb_df.groupby('Metropolitan area')['MLB W/L'].mean()
    
    
    ##### MERGING SERIES TO DATAFRAME #####
    
    df = cities.merge(MLB, left_index=True, right_index=True)
    
    
    ##### SELECTING DATA FOR PEARSONR FUNCTION #####
    
    population_by_region = df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['MLB W/L'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [157]:
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [160]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    
    # Reading data
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nfl_df = pd.read_csv("assets/nfl.csv")
    
    
    ##### CLEANING CITIES DATA #####
    
    # Removing special characters
    cities.replace('\[([^)]*)\]', '', regex=True, inplace=True)
    
    # Including ' ' into team names
    cities['NHL'] = cities['NHL'].apply(lambda x: ' '.join(re.compile('[A-Z][a-z]+').findall(x)))
    
    # Setting cities as index
    cities.set_index('Metropolitan area', inplace=True)
    
    # Converting str to int
    cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype(int)
    
    
    ##### CLEANING NFL #####
    
    # Filtering data
    nfl_df = nfl_df.loc[nfl_df['year'] == 2018].copy()
    nfl_df = nfl_df[nfl_df['W'].str.isnumeric()]
    
    # Converting str to int
    nfl_df['W'] = nfl_df['W'].astype(int)
    nfl_df['L'] = nfl_df['L'].astype(int)
    
    # Removing special character
    nfl_df.replace('\*', '', regex=True, inplace=True)
    nfl_df.replace('\+', '', regex=True, inplace=True)

    # Creating Metropolitan area column
    lst = nfl_df['team'].to_list()

    for e,i in enumerate(lst):
        st = i.split(' ')
        if len(st) > 2:
            lst[e] = st[0] + ' ' + st[1]
        else:
            lst[e] = st[0]

    nfl_df['Metropolitan area'] = lst

    Metropolitan_area_dict = {'Carolina':'Charlotte',
                              'San Francisco':'San Francisco Bay Area',
                              'Minnesota':'Minneapolis–Saint Paul',
                              'Miami':'Miami–Fort Lauderdale',
                              'Tennessee':'Nashville',
                              'Tampa Bay':'Tampa Bay Area',
                              'Washington':'Washington, D.C.',
                              'New England':'Boston',
                              'Arizona':'Phoenix',
                              'New York':'New York City',
                              'Oakland':'San Francisco Bay Area',
                              'Dallas':'Dallas–Fort Worth'}

    nfl_df['Metropolitan area'].replace(Metropolitan_area_dict, inplace=True)
    
    # Creating measure NHL ratio
    nfl_df['NFL W/L'] = nfl_df['W'] / (nfl_df['W'] + nfl_df['L'])
    
    # Grouping by data in a Series with Metropolitan area index
    NFL = nfl_df.groupby('Metropolitan area')['NFL W/L'].mean()


    ##### MERGING SERIES TO DATAFRAME #####
    
    df = cities.merge(NFL, left_index=True, right_index=True)
    
    
    ##### SELECTING DATA FOR PEARSONR FUNCTION #####
    
    
    population_by_region = df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['NFL W/L'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [159]:
nfl_correlation()

(0.004922112149349428, 0.9797833458363694)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [162]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    
    ##### READING DATA #####

    nba_df=pd.read_csv("assets/nba.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    mlb_df=pd.read_csv("assets/mlb.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
        
    
    ##### CLEANING NBA DATA #####
    
    # Removing special character
    nba_df['team'].replace('\*', '', regex=True, inplace=True)
    nba_df['team'].replace('\(([^\)]+)\)', '', regex=True, inplace=True)
    
    # Filtering data
    nba_df = nba_df.loc[nba_df['year'] == 2018]
    
    # Converting str to int
    nba_df['W'] = nba_df['W'].astype(int)
    nba_df['L'] = nba_df['L'].astype(int)

    # Creating Metropolitan area column
    lst = nba_df['team']
    lst = lst.str.split(' ')

    for e,i in enumerate(lst):
        if len(i) > 2:
            lst[e] = i[0] + ' ' + i[1]
        else:
            lst[e] = i[0]

    nba_df['Metropolitan area'] = lst

    Metropolitan_area_dict = {'Brooklyn':'New York City',
                              'Dallas':'Dallas–Fort Worth',
                              'Golden State':'San Francisco Bay Area',
                              'Indiana':'Indianapolis',
                              'Dallas':'Dallas–Fort Worth',
                              'Miami':'Miami–Fort Lauderdale',
                              'Minnesota':'Minneapolis–Saint Paul',
                              'New York':'New York City',
                              'Utah':'Salt Lake City',
                              'Portland Trail':'Portland',
                              'Washington':'Washington, D.C.'}

    nba_df['Metropolitan area'].replace(Metropolitan_area_dict, inplace=True)
    
    # Creating measure NBA ratio
    nba_df['NBA'] = nba_df['W'] / (nba_df['W'] + nba_df['L'])
    
    # Grouping by data in a Series with Metropolitan area index
    NBA = nba_df.groupby('Metropolitan area')['NBA'].mean()
    
        
    ##### CLEANING NHL #####
    
    # Removing special character
    nhl_df['team'].replace('\*', '', regex=True, inplace=True)
    
    # Filtering data
    nhl_df = nhl_df.loc[nhl_df['year'] == 2018]
    nhl_df = nhl_df[nhl_df['GP'].str.isnumeric()]
    
    # Creating Metropolitan area column
    nhl_df['Metropolitan area'] = nhl_df['team'].str.split(' ', expand=True)[0]
    Metropolitan_area_dict = {'Anaheim':'Los Angeles','Arizona':'Phoenix','Carolina':'Raleigh','Colorado':'Denver','Dallas':'Dallas–Fort Worth','Florida':'Miami–Fort Lauderdale','Los':'Los Angeles','Minnesota':'Minneapolis–Saint Paul','New':'New York City','San':'San Francisco Bay Area','St.':'St. Louis','Tampa':'Tampa Bay Area','Vegas':'Las Vegas','Washington':'Washington, D.C.'}
    nhl_df['Metropolitan area'].replace(Metropolitan_area_dict, inplace=True)
    
    # Converting str to int
    nhl_df['W'] = nhl_df['W'].astype(int)
    nhl_df['L'] = nhl_df['L'].astype(int)

    # Creating measure NHL ratio
    nhl_df['NHL'] = nhl_df['W'] / (nhl_df['W'] + nhl_df['L'])

    # Grouping by data in a Series with Metropolitan area index
    NHL = nhl_df.groupby('Metropolitan area')['NHL'].mean()
    
    
    ##### CLEANING MLB #####
    
    # Filtering data
    mlb_df = mlb_df.loc[mlb_df['year'] == 2018]
    
    # Creating Metropolitan area column
    lst = mlb_df['team'].str.split(' ')

    for e,item in enumerate(lst):
        if len(item) > 2:
            lst[e] = item[0] + ' ' + item[1]
        else:
            lst[e] = item[0]
    
    mlb_df['Metropolitan area'] = lst

    Metropolitan_area_dict = {'San Francisco':'San Francisco Bay Area',
                              'Minnesota':'Minneapolis–Saint Paul',
                              'Toronto Blue':'Toronto',
                              'Miami':'Miami–Fort Lauderdale',
                              'Chicago White':'Chicago',
                              'Tampa Bay':'Tampa Bay Area',
                              'Washington':'Washington, D.C.',
                              'Boston Red':'Boston',
                              'Texas':'Dallas–Fort Worth',
                              'Arizona':'Phoenix',
                              'Colorado':'Denver',
                              'New York':'New York City',
                              'Oakland':'San Francisco Bay Area'}

    mlb_df['Metropolitan area'].replace(Metropolitan_area_dict, inplace=True)
    
    # Creating measure MLB ratio
    mlb_df['MLB'] = mlb_df['W'] / (mlb_df['W'] + mlb_df['L'])
    
    # Grouping by data in a Series with Metropolitan area index
    MLB = mlb_df.groupby('Metropolitan area')['MLB'].mean()
    

    ##### CLEANING NFL #####
    
    # Filtering data
    nfl_df = nfl_df.loc[nfl_df['year'] == 2018].copy()
    nfl_df = nfl_df[nfl_df['W'].str.isnumeric()]
    
    # Converting str to int
    nfl_df['W'] = nfl_df['W'].astype(int)
    nfl_df['L'] = nfl_df['L'].astype(int)
    
    # Removing special character
    nfl_df.replace('\*', '', regex=True, inplace=True)
    nfl_df.replace('\+', '', regex=True, inplace=True)

    # Creating Metropolitan area column
    lst = nfl_df['team'].to_list()

    for e,i in enumerate(lst):
        st = i.split(' ')
        if len(st) > 2:
            lst[e] = st[0] + ' ' + st[1]
        else:
            lst[e] = st[0]

    nfl_df['Metropolitan area'] = lst

    Metropolitan_area_dict = {'Carolina':'Charlotte',
                              'San Francisco':'San Francisco Bay Area',
                              'Minnesota':'Minneapolis–Saint Paul',
                              'Miami':'Miami–Fort Lauderdale',
                              'Tennessee':'Nashville',
                              'Tampa Bay':'Tampa Bay Area',
                              'Washington':'Washington, D.C.',
                              'New England':'Boston',
                              'Arizona':'Phoenix',
                              'New York':'New York City',
                              'Oakland':'San Francisco Bay Area',
                              'Dallas':'Dallas–Fort Worth'}

    nfl_df['Metropolitan area'].replace(Metropolitan_area_dict, inplace=True)
    
    # Creating measure NHL ratio
    nfl_df['NFL'] = nfl_df['W'] / (nfl_df['W'] + nfl_df['L'])
    
    # Grouping by data in a Series with Metropolitan area index
    NFL = nfl_df.groupby('Metropolitan area')['NFL'].mean()
    
    
    ##### RUNNING TTEST_REL FOR EACH RELATION #####
    
    nba_nhl = pd.merge(NBA, NHL, left_index=True, right_index=True)
    nba_nhl_p_values = stats.ttest_rel(nba_nhl["NBA"], nba_nhl["NHL"])[1]    
    
    nba_mlb = pd.merge(NBA, MLB, left_index=True, right_index=True)
    nba_mlb_p_values = stats.ttest_rel(nba_mlb["NBA"], nba_mlb["MLB"])[1]
    
    nba_nfl = pd.merge(NBA, NFL, left_index=True, right_index=True)
    nba_nfl_p_values = stats.ttest_rel(nba_nfl["NBA"], nba_nfl["NFL"])[1]
    
    nhl_mlb = pd.merge(NHL, MLB, left_index=True, right_index=True)
    nhl_mlb_p_values = stats.ttest_rel(nhl_mlb["NHL"], nhl_mlb["MLB"])[1]
    
    nhl_nfl = pd.merge(NHL, NFL, left_index=True, right_index=True)
    nhl_nfl_p_values = stats.ttest_rel(nhl_nfl["NHL"], nhl_nfl["NFL"])[1]
    
    mlb_nfl = pd.merge(MLB, NFL, left_index=True, right_index=True)
    mlb_nfl_p_values = stats.ttest_rel(mlb_nfl["MLB"], mlb_nfl["NFL"])[1]
    
    
    ##### CREATING P_VALUE DATAFRAME #####
    
    p_values_dict = {'NBA':{'NBA':np.nan,'NHL':nba_nhl_p_values,'MLB':nba_mlb_p_values,'NFL':nba_nfl_p_values},
                     'NHL':{'NBA':nba_nhl_p_values,'NHL':np.nan,'MLB':nhl_mlb_p_values,'NFL':nhl_nfl_p_values},
                     'MLB':{'NBA':nba_mlb_p_values,'NHL':nhl_mlb_p_values,'MLB':np.nan,'NFL':mlb_nfl_p_values},
                     'NFL':{'NBA':nba_nfl_p_values,'NHL':nhl_nfl_p_values,'MLB':mlb_nfl_p_values,'NFL':np.nan}}
    
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan

    p_values = pd.DataFrame(p_values_dict)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    
    return p_values

In [161]:
sports_team_performance()

,NBA,NHL,MLB,NFL
NBA,NaN,0.022297,0.950540,0.941792
NHL,0.022297,NaN,0.000708,0.030883
MLB,0.950540,0.000708,NaN,0.802069
NFL,0.941792,0.030883,0.802069,NaN
